# Python SQLite

### https://gist.github.com/imShakil/478859e7a1aba8e08057535a062e6537

In [1]:
import sqlite3

#### Create a database (or to connect to an existing one)

In [4]:
db = sqlite3.connect("testing.db")
# a .db file has just been created

#### .cursor(); CREATE TABLE

In [5]:
# need a cursor object to continue
cur = db.cursor()

In [6]:
# create a table
cur.execute(
            """CREATE TABLE IF NOT EXISTS users(
                id INTEGER PRIMARY KEY,
                name TEXT,
                phone TEXT unique)
                """)

In [8]:
# need to commit
db.commit()

#### INSERT INTO

In [10]:
cur.execute(
            """INSERT INTO users(id, name, phone) 
                VALUES(?,?,?)""",
            (123,"Karina", "55303171"))

In [11]:
db.commit()

In [13]:
# can insert a list of tubles

import random

newdata = [
    
    (
        random.randint(100,900),
        "Robot" + str(i),
        str(random.randint(10000000,99999999))
    )
    for i in range(10)
]

In [15]:
newdata

[(776, 'Robot0', '26478576'),
 (868, 'Robot1', '95808426'),
 (628, 'Robot2', '82396899'),
 (585, 'Robot3', '45885223'),
 (255, 'Robot4', '74795794'),
 (269, 'Robot5', '13740200'),
 (212, 'Robot6', '73584595'),
 (231, 'Robot7', '13111275'),
 (106, 'Robot8', '28027617'),
 (856, 'Robot9', '98599725')]

In [17]:
# .executemany() !
cur.executemany(
                """INSERT INTO users(id, name, phone) VALUES(?,?,?)""",
                newdata)

In [18]:
db.commit()

#### SELECT FROM (Retrieving data from database)

#### need to use with .fetchone() / .fetchmany() / .fetchall()

In [26]:
cur.execute("""SELECT id, name, phone FROM users""")

In [27]:
# .fetchmany() and .fetchone()
print(cur.fetchone())
print(cur.fetchmany(3))

(106, 'Robot8', '28027617')
[(123, 'Karina', '55303171'), (212, 'Robot6', '73584595'), (231, 'Robot7', '13111275')]


In [28]:
# .fetchall()
all_users = cur.fetchall()
print(all_users)

[(255, 'Robot4', '74795794'), (269, 'Robot5', '13740200'), (585, 'Robot3', '45885223'), (628, 'Robot2', '82396899'), (776, 'Robot0', '26478576'), (856, 'Robot9', '98599725'), (868, 'Robot1', '95808426')]


Note that the cursor will go down and would not repeat the items that had already been fetched

In [29]:
print(cur.fetchone())

None


If you want to fetch again, need to execute the SELECT FROM statement again

In [30]:
cur.execute("""SELECT id, name, phone FROM users""")
all_users = cur.fetchall()
print(all_users)

[(106, 'Robot8', '28027617'), (123, 'Karina', '55303171'), (212, 'Robot6', '73584595'), (231, 'Robot7', '13111275'), (255, 'Robot4', '74795794'), (269, 'Robot5', '13740200'), (585, 'Robot3', '45885223'), (628, 'Robot2', '82396899'), (776, 'Robot0', '26478576'), (856, 'Robot9', '98599725'), (868, 'Robot1', '95808426')]


In [0]:
db.commit()

#### WHERE

In [35]:
cur.execute("""SELECT id, name, phone FROM users WHERE id = 123""")

In [36]:
print(cur.fetchall())

[(123, 'Karina', '55303171')]


In [0]:
db.commit()

#### UPDATE SET WHERE

In [37]:
cur.execute("""UPDATE users SET phone = ? WHERE id = ?""", ("999", 123))

In [38]:
# looking at the results
cur.execute("""SELECT id, name, phone FROM users WHERE id = 123""")
print(cur.fetchall())

[(123, 'Karina', '999')]


In [0]:
db.commit()

#### DELETE FROM

In [41]:
cur.execute("""DELETE FROM users WHERE name = 'Karina'""")

In [42]:
# looking at the results
cur.execute("""SELECT id, name, phone FROM users WHERE id = 123""")
print(cur.fetchall())

[]


In [43]:
db.commit()

#### DROP TABLE

In [44]:
cur.execute("""DROP TABLE IF EXISTS users""")

In [45]:
db.commit()

## Class teaching - loading a textfile into a database

In [46]:
# Create a new database
conn = sqlite3.connect("variant.db")
c = conn.cursor()

In [47]:
# Create a new table called variants
c.execute("""CREATE TABLE IF NOT EXISTS 
                variants(
                        id INTEGER PRIMARY KEY,
                        chrom TEXT,
                        start INTEGER,
                        end INTEGER,
                        strand TEXT,
                        rsid TEXT)""")

In [49]:
conn.commit()

In [51]:
# Loading the textfile
data = []
with open("variants.txt") as fh:
    for line in fh:
        data.append(tuple(line.strip().split("\t")))
print(data)

[('chr10', '114808901', '114808902', '+', 'rs12255372'), ('chr9', '22125502', '22125503', '+', 'rs1333049'), ('chr3', '46414946', '46414978', '+', 'rs333'), ('chr2', '136608645', '136608646', '-', 'rs4988235')]


In [52]:
# Insert into the database
c.executemany("""INSERT INTO variants(chrom, start, end, strand, rsid) VALUES(?,?,?,?,?)""", data)

In [55]:
# Looking at the database
c.execute("""SELECT id, chrom, start, end, strand, rsid FROM variants""")
print(c.fetchall()) # note that the id will be generated automatically

[(1, 'chr10', 114808901, 114808902, '+', 'rs12255372'), (2, 'chr9', 22125502, 22125503, '+', 'rs1333049'), (3, 'chr3', 46414946, 46414978, '+', 'rs333'), (4, 'chr2', 136608645, 136608646, '-', 'rs4988235')]


In [56]:
conn.commit()

## Class teaching - loading a *.json file into the *.db

In [57]:
# Loading the *.json file first

import json

jdata = open("roster_data_sample.json").read()
data = json.loads(jdata)

In [59]:
print(data[:5])

[['Charley', 'si110', 1], ['Mea', 'si110', 0], ['Hattie', 'si110', 0], ['Lyena', 'si110', 0], ['Keziah', 'si110', 0]]


In [60]:
# Create a new database and new table

conn = sqlite3.connect("roster.db")
c = conn.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS member(
                id INTEGER,
                course_id INTEGER,
                role INTEGER,
                PRIMARY KEY (id, course_id))""")

conn.commit()

In [61]:
# convert the data into a list of tuples

newdata = [tuple(i) for i in data]
print(newdata[:5])

[('Charley', 'si110', 1), ('Mea', 'si110', 0), ('Hattie', 'si110', 0), ('Lyena', 'si110', 0), ('Keziah', 'si110', 0)]


In [66]:
# insert the data into the table

c.executemany("""INSERT INTO member (id, course_id, role) VALUES(?,?,?)""", newdata)

In [68]:
# checking out on the results

c.execute("""SELECT id, course_id, role FROM member""")
print(c.fetchmany(10))

[('Charley', 'si110', 1), ('Mea', 'si110', 0), ('Hattie', 'si110', 0), ('Lyena', 'si110', 0), ('Keziah', 'si110', 0), ('Ellyce', 'si110', 0), ('Thalia', 'si110', 0), ('Meabh', 'si110', 0), ('Aria', 'si110', 0), ('Reena', 'si110', 0)]


In [69]:
conn.commit()